In [1]:
import os
from IPython.core.display import HTML

# with open('creative_commons.txt', 'r') as f:
    #html = f.read()

html = "Use as you wish!"
    
name = '2015-02-02-cartopy_folium_shapefile'

html = '''
<small>
<p> This post was written as an IPython notebook.
 It is available for <a href='https://ocefpaf.github.com/python4oceanographers/downloads/notebooks/%s.ipynb'>download</a>
 or as a static <a href='https://nbviewer.ipython.org/url/ocefpaf.github.com/python4oceanographers/downloads/notebooks/%s.ipynb'>html</a>.</p>
<p></p>
%s''' % (name, name, html)

%matplotlib inline
from matplotlib import style
style.use('ggplot')

from datetime import datetime

title = 'Plotting shapefiles with cartopy and folium'
hour = datetime.utcnow().strftime('%H:%M')
comments="true"

date = '-'.join(name.split('-')[:3])
slug = '-'.join(name.split('-')[3:])

metadata = dict(title=title,
                date=date,
                hour=hour,
                comments=comments,
                slug=slug,
                name=name)

markdown = """Title: {title}
date:  {date} {hour}
comments: {comments}
slug: {slug}

{{% notebook {name}.ipynb cells[1:] %}}
""".format(**metadata)

content = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, '{}.md'.format(name)))
with open('{}'.format(content), 'w') as f:
    f.writelines(markdown)

First we will use cartopy's
[shapereader](http://scitools.org.uk/cartopy/docs/latest/tutorials/using_the_shapereader.html) 
to download (and cache) states shapefile with 50 meters resolution from the 
[NaturalEarth](http://www.naturalearthdata.com/).

In [2]:
from cartopy.io import shapereader


kw = dict(resolution='50m', category='cultural',
          name='admin_1_states_provinces')

states_shp = shapereader.natural_earth(**kw)
shp = shapereader.Reader(states_shp)

Now let's create a list of a few states that we want to highlight.

(The `unicode_literals` is needed to compare the Unicode names with the decoded
names from latin-1.  Argh!!! One day all of python will be Python3 and this
ordeal will be over.)

In [3]:
from __future__ import unicode_literals

states = ('Minas Gerais', 'Mato Grosso', 'Goiás',
          'Bahia', 'Rio Grande do Sul', 'São Paulo')

Let's plot it all with cartopy.

Cartopy's maps are great, but they are not interactive.  We can fix that by
plotting the same data over a
[folium](https://github.com/python-visualization/folium) `Map` instance.

The trick is to save the
[shapefile](http://en.wikipedia.org/wiki/Shapefile) as a
[GeoJSON](http://geojson.org/) and plot it with folium's `.geo_json`.

In [4]:
import os
import shapefile
from json import dumps


def shape2json(fname, outfile="states.json", country='Brazil'):
    reader = shapefile.Reader(fname)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    data = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        if country in sr.record[field_names.index('admin')]:
            name = sr.record[field_names.index('name')].decode('latin-1')
            if name in states:
                data.append(dict(type="Feature", geometry=geom, properties=atr))
            
    keys = ['abbrev', 'name', 'name_alt']
    for b in data:
        for key in keys:
            b['properties'][key] = b['properties'][key].decode('latin-1')

    with open(outfile, "w") as geojson:
        geojson.write(dumps({"type": "FeatureCollection",
                             "features": data}, indent=2) + "\n")


shape = '50m_admin_1_states_provinces.shp'
cartopy_cache = '.local/share/cartopy/shapefiles/natural_earth/cultural/'
fname = os.path.join(os.path.expanduser('~'), cartopy_cache, shape)

shape2json(fname, outfile="states.json", country='Brazil')

ShapefileException: Unable to open /home/tony/.local/share/cartopy/shapefiles/natural_earth/cultural/50m_admin_1_states_provinces.dbf or /home/tony/.local/share/cartopy/shapefiles/natural_earth/cultural/50m_admin_1_states_provinces.shp.

In [ ]:
import folium
import numpy as np
from IPython.display import IFrame


def inline_map(m, width=650, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed

bbox = [-82, -32, -45, 10]

lon_center, lat_center = np.array(bbox).reshape(2, 2).mean(axis=0)
mapa = folium.Map(width=650, height=500, zoom_start=4,
                  location=[-15, -50])

mapa.geo_json(geo_path='states.json', fill_color='none', line_color='Orange')
inline_map(mapa)

That was easy!  Check
[here](http://folium.readthedocs.org/en/latest/#data-mapping-geojson-and-topojson)
to learn more about GeoJSONs plotting with folium.

In [ ]:
HTML(html)